In [ ]:
class PatchSave(EOTask):
    def __init__(self, patch):
        self.patch = patch       
            
    def execute(self, eopatch):
        eopatch = patch
        return eopatch    
    
patch_out_dnbr = './eopatches_dnbr/' if use_smaller_patches else './eopatches_dnbr/'
if not os.path.isdir(patch_out_dnbr):
    os.makedirs(patch_out_dnbr)
    
patch_save = PatchSave(patch)    
save_dnbr = SaveToDisk(path_out_dnbr, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

In [ ]:
workflow_save = LinearWorkflow(
    patch_save,     # add rasterisation task, if you want to use land_cover_map
    save_dnbr
)

# Let's visualize it
workflow_save.dependency_graph()

In [ ]:
%%time

# define additional parameters of the workflow
execution_args = []
for idx, bbox in enumerate(bbox_list[patchIDs]):
    execution_args.append({
        save_dnbr: {'eopatch_folder': 'eopatch_{}'.format(idx)}
    })

executor = EOExecutor(workflow_save, execution_args, save_logs=True)
executor.run(workers=5, multiprocess=False)

executor.make_report()

In [ ]:
# upgrade this for save dnbr to eo-patch

# RasterToVector Задача для преобразования растровой маски в векторную.

In [ ]:
# # TASK TO EXPORT TIFF
# load = EOPatch.load('./result_patch')

# export_tiff = ExportToTiff((FeatureType.DATA, 'dNBR'))
# tiff_location = './result_tiff'
# if not os.path.isdir(tiff_location):
#     os.makedirs(tiff_location)
    
# execution_args = [{'load' : {'eopatch_folder': 'result_patch'}, 
#                   'export_tiff': {'filename': '/result_tiff1.tiff'}}]   
    
# workflow_export = LinearWorkflow(load, export_tiff)    

# executor = EOExecutor(workflow_export, execution_args)            
# executor.run(workers=2, multiprocess=False)
# executor.make_report()

In [ ]:
# # array of shape (h, w)
# dst_shape = delta_nbr_1.shape
# bbox=[27.48, 51.75, 27.57, 51.64]

# # bbox is the bounding box BBox instance used in Wms/WcsRequest 
# dst_transform = rasterio.transform.from_bounds(*roi_bbox.get_lower_left(), *roi_bbox.get_upper_right(),
#                                                width=dst_shape[1], height=dst_shape[0])
# dst_crs = 'EPSG:4326'
# # dst_crs = {'init': CRS.ogc_string(bbox.crs)}

# # Write it out to a file.
# with rasterio.open('result_file.tif', 'w', driver='GTiff',
#                               width=dst_shape[1], height=dst_shape[0],
#                               count=1, dtype=np.uint8, nodata=0,
#                               transform=dst_transform, crs=dst_crs) as dst:
#     dst.write(delta_nbr_1.astype(np.uint8))
      

In [ ]:
# # array of shape (h, w)
# dst_shape = delta_nbr_1.shape
# bbox=[27.48, 51.75, 27.57, 51.64]

# # bbox is the bounding box BBox instance used in Wms/WcsRequest 
# dst_transform = rasterio.transform.from_bounds(*roi_bbox.get_lower_left(), *roi_bbox.get_upper_right(),
#                                                width=dst_shape[1], height=dst_shape[0])
# dst_crs = 'EPSG:4326'
# # dst_crs = {'init': CRS.ogc_string(bbox.crs)}

# # Write it out to a file.
# with rasterio.open('result_file.tif', 'w', driver='GTiff',
#                               width=dst_shape[1], height=dst_shape[0],
#                               count=1, dtype=np.uint8, nodata=0,
#                               transform=dst_transform, crs=dst_crs) as dst:
#     dst.write(delta_nbr_1.astype(np.uint8))

In [ ]:
# result_export = workflow_export.execute()

In [ ]:
for i in range(1,2):
    print(i)

In [ ]:
with rio.open('./exemple_tiff/example.tiff') as src:
    naip_data_ras = src.read()
    naip_meta = src.profile
    
    
naip_meta    

In [ ]:
naip_transform = naip_meta["transform"]
naip_crs = naip_meta["crs"]
dst_shape = delta_nbr_1.shape
bbox=[27.48, 51.75, 27.57, 51.64]

# View spatial attributes
naip_transform, naip_crs

In [ ]:
# with rasterio.open('result_file.tiff', 'w', driver='GTiff', width=dst_shape[1], height=dst_shape[0],
#                               count=1, dtype=np.uint8, nodata=0,
#                               transform=dst_transform, crs=dst_crs) as dst:
#     dst.write(delta_nbr_1.astype(np.uint8), 1)    

In [ ]:
# # Write your the ndvi raster object
# with rio.open('result_file.tiff', 'w', **naip_meta) as dst:
#     dst.write(delta_nbr_1.astype(np.float64), 1)

In [ ]:
min = np.amin(delta_nbr_1)
max = np.amax(delta_nbr_1)
print (min)
print (max)

In [ ]:
def diverse(array):             # returns the amplitude of the values
    min = np.amin(array)
    max = np.amax(array)
    
    if min <= 0 and max >= 0:
        diverse = abs(min) + max
    elif min >= 0 and max >= 0:
        diverse = max - min
    elif min <= 0 and max <= 0:
        diverse = min - max
    
    return(diverse)

In [ ]:
a = diverse(delta_nbr_1)
print(a)

In [ ]:
def normal(array):             # приведение значений массива к 8-битным знаечниям (0,..,255)
    min = np.amin(array)
    max = np.amax(array)
    
    if min <= 0 and max >= 0:
        diverse = abs(min) + max
        
        positive_values = diverse + min      # получение положительного промежутка
#        negative_values = diverse - positive_values   # получение отрицательного промежутка
        
        positive_values_part = positive_values / diverse    # получение доли положительного промежутка
        negative_values_part = 1 - positive_values_part     # получение доли отрицательного промежутка
        
        pos8bit = round(positive_values_part * 256)
        neg8bit = 256 - pos8bit
        
        neg_step = (min - 0.00001) / neg8bit    # получение шага яркости равного изменению на 1
        
        #  необходимо понять, как реализовать проверку каждого отдельного значения в массиве
        #  возможно,стоит попробовать разделить 3-хмерный массив на одномерные 
        #
        
    elif min >= 0 and max >= 0:
        diverse = max - min
        
        
    elif min <= 0 and max <= 0:
        diverse = min - max
        
    return (positive_values_part, negative_values_part, pos8bit, neg8bit, neg_step)    